# Literature exploration by term co-occurrance analysis

In this example we illustrate how network analytics can be used for literature exploration. 

The input dataset contains occurrences of different terms in paragraphs of scientific articles previously extracted by means of a Named Entity Recognition (NER) model. This dataset is transformed into three co-occurrence networks: representing paper-, section- and paragraph-level co-occurrence relation between terms. The term relations in the above-mentioned networks are quantified using mutual-information-based scores (pointwise mutual information and its normalized version).

The networks are further analysed using classical tools from complex networks: we find various centrality measures characterizing the importance of extracted terms, we detect term communities representing denesely connected clusters of terms and finally we illustrate how the algorithms for finding shortest paths and minimum spanning trees can be used to perform guided search in networks.

In [8]:
import networkx as nx
import pandas as pd
import numpy as np

In [9]:
from bluegraph.core import (PandasPGFrame,
                            pretty_print_paths,
                            pretty_print_tripaths,
                            graph_elements_from_paths)
from bluegraph.preprocess.generators import CooccurrenceGenerator

from bluegraph.backends.graph_tool import GTMetricProcessor, GTPathFinder
from bluegraph.backends.graph_tool import graph_tool_to_pgframe

from bluegraph.backends.networkx import NXMetricProcessor, NXPathFinder

from bluegraph.backends.stellargraph import StellarGraphNodeEmbedder

## Prepare entity-occurrence data property graph

The input data is given by occurrences of different entities in specific paragraphs of scientific articles.

In [10]:
mentions = pd.read_csv("data/literature_NER_example.csv")

Every paragraph is identified using the format `<paper_id>:<section_id>:<paragraph_id>`. From this data we will extract occurrences in distinct papers/sections/paragraphs as follows:

In [11]:
# Extract unique paper/seciton/paragraph identifiers
mentions["paper"] = mentions["occurrence"].apply(
    lambda x: x.split(":")[0])
mentions["section"] = mentions["occurrence"].apply(
    lambda x: ":".join([x.split(":")[0], x.split(":")[1]]))

mentions = mentions.rename(columns={"occurrence": "paragraph"})

We, first, create an empty property graph

In [12]:
graph = PandasPGFrame()

Then we add nodes for unique entities and papers

In [13]:
entity_nodes = mentions["entity"].unique()
graph.add_nodes(entity_nodes)
graph.add_node_types({n: "Entity" for n in entity_nodes})

paper_nodes = mentions["paper"].unique()
graph.add_nodes(paper_nodes)
graph.add_node_types({n: "Paper" for n in paper_nodes})

In [14]:
mentions

,entity,paragraph,paper,section
0,ace inhibitor,184360:Combined Therapeutic Potential Targetin...,184360,184360:Combined Therapeutic Potential Targetin...
1,ace inhibitor,184360:Caption:68,184360,184360:Caption
2,ace inhibitor,184360:Aceis And Arbs ::: Therapeutic Potentia...,184360,184360:Aceis And Arbs
3,ace inhibitor,197804:Caption:71,197804,197804:Caption
4,ace inhibitor,184360:Conclusion:62,184360,184360:Conclusion
...,...,...,...,...
1552,virus,214924:Abstract:1,214924,214924:Abstract
1553,virus,179426:Role Of Dpp4 Enzyme And Dpp4 Inhibitors...,179426,179426:Role Of Dpp4 Enzyme And Dpp4 Inhibitors
1554,virus,184360:Combined Therapeutic Potential Targetin...,184360,184360:Combined Therapeutic Potential Targetin...
1555,virus,197804:Introduction:2,197804,197804:Introduction


In [15]:
graph.nodes(raw_frame=True)

,@type
@id,
ace inhibitor,Entity
acetaminophen,Entity
acute lung injury,Entity
acute respiratory distress syndrome,Entity
adenosine,Entity
...,...
78884,Paper
35198,Paper
139943,Paper


We now add edges from entities to the papers they occur in

In [16]:
occurrence_edges = mentions.groupby(by=["entity", "paper"]).aggregate(set)

In [17]:
occurrence_edges

paragraph  \
entity            paper                                                       
ace inhibitor     184360  {184360:Caption:68, 184360:Conclusion:62, 1843...   
                  197804  {197804:Caption:72, 197804:Caption:71, 197804:...   
acetaminophen     179426  {179426:Blood Glucose Monitoring ::: Special A...   
                  197804  {197804:Management Of Children And Young Peopl...   
acute lung injury 179426  {179426:Role Of Ace/Arbs ::: Special Aspects O...   
...                                                                     ...   
virus             184360  {184360:Abstract:1, 184360:Sdpp4 As Soluble De...   
                  197804      {197804:Introduction:2, 197804:Discussion:44}   
                  211125                             {211125:Discussion:25}   
                  211373  {211373:Introduction:3, 211373:Introduction:6,...   
                  214924  {214924:Abstract:1, 214924:The Interplay Betwe...   

                                                                    section  
entity            paper                                                      
ace inhibitor     184360  {184360:Conclusion, 184360:Caption, 184360:Ace...  
                  197804  {197804:Caption, 197804:Management Of Children...  
acetaminophen     179426                 {179426:Blood Glucose Monitoring }  
                  197804  {197804:Discussion, 197804:Management Of Child...  
acute lung injury 179426                         {179426:Role Of Ace/Arbs }  
...                                                                     ...  
virus             184360  {184360:Anti-Dpp4 Vaccine , 184360:Gliptins , ...  
                  197804           {197804:Introduction, 197804:Discussion}  
                  211125                                {211125:Discussion}  
                  211373                              {211373:Introduction}  
                  214924  {214924:The Immune Response To Sars-Cov-2 , 21...  

[551 rows x 2 columns]

In [18]:
graph.add_edges(occurrence_edges.index)
graph.add_edge_types({e: "OccursIn" for e in occurrence_edges.index})

In [19]:
occurrence_edges.index = occurrence_edges.index.rename(["@source_id", "@target_id"])

In [20]:
graph.add_edge_properties(occurrence_edges["paragraph"])
graph.add_edge_properties(occurrence_edges["section"])

In [21]:
graph.edges(raw_frame=True)

@type  \
@source_id        @target_id             
ace inhibitor     184360      OccursIn   
                  197804      OccursIn   
acetaminophen     179426      OccursIn   
                  197804      OccursIn   
acute lung injury 179426      OccursIn   
...                                ...   
virus             184360      OccursIn   
                  197804      OccursIn   
                  211125      OccursIn   
                  211373      OccursIn   
                  214924      OccursIn   

                                                                      paragraph  \
@source_id        @target_id                                                      
ace inhibitor     184360      {184360:Caption:68, 184360:Conclusion:62, 1843...   
                  197804      {197804:Caption:72, 197804:Caption:71, 197804:...   
acetaminophen     179426      {179426:Blood Glucose Monitoring ::: Special A...   
                  197804      {197804:Management Of Children And Young Peopl...   
acute lung injury 179426      {179426:Role Of Ace/Arbs ::: Special Aspects O...   
...                                                                         ...   
virus             184360      {184360:Abstract:1, 184360:Sdpp4 As Soluble De...   
                  197804          {197804:Introduction:2, 197804:Discussion:44}   
                  211125                                 {211125:Discussion:25}   
                  211373      {211373:Introduction:3, 211373:Introduction:6,...   
                  214924      {214924:Abstract:1, 214924:The Interplay Betwe...   

                                                                        section  
@source_id        @target_id                                                     
ace inhibitor     184360      {184360:Conclusion, 184360:Caption, 184360:Ace...  
                  197804      {197804:Caption, 197804:Management Of Children...  
acetaminophen     179426                     {179426:Blood Glucose Monitoring }  
                  197804      {197804:Discussion, 197804:Management Of Child...  
acute lung injury 179426                             {179426:Role Of Ace/Arbs }  
...                                                                         ...  
virus             184360      {184360:Anti-Dpp4 Vaccine , 184360:Gliptins , ...  
                  197804               {197804:Introduction, 197804:Discussion}  
                  211125                                    {211125:Discussion}  
                  211373                                  {211373:Introduction}  
                  214924      {214924:The Immune Response To Sars-Cov-2 , 21...  

[551 rows x 3 columns]

## Generate co-occurrence networks

We will generate co-occurrence networks for difference occurrence factors (paper/section/paragraph), i.e. an edge between a pair of entities is added if they co-occur in the same paper/section/paragraph/

### Paper-based co-occurrence

In [22]:
gen = CooccurrenceGenerator(graph)
paper_cooccurrence_edges = gen.generate_from_edges(
     "OccursIn", compute_statistics=["frequency", "ppmi", "npmi"])

Examining 12246 pairs of terms for co-occurrence...


In [23]:
paper_cooccurrence_edges["@type"] = "CoOccursWith"
paper_cooccurrence_edges

common_factors  \
@source_id      @target_id                                                                      
ace inhibitor   acetaminophen                                                        {197804}   
                acute lung injury                                            {197804, 184360}   
                acute respiratory distress syndrome                          {197804, 184360}   
                adenosine                                                            {184360}   
                adipose tissue                                               {197804, 184360}   
...                                                                                       ...   
viral           viral infection                              {211125, 214924, 211373, 184360}   
                virus                                {179426, 184360, 211125, 214924, 211373}   
viral entry     viral infection                                                      {214924}   
                virus                                                        {214924, 179426}   
viral infection virus                                        {211125, 214924, 211373, 184360}   

                                                     frequency      ppmi  \
@source_id      @target_id                                                 
ace inhibitor   acetaminophen                                1  2.321928   
                acute lung injury                            2  2.321928   
                acute respiratory distress syndrome          2  1.736966   
                adenosine                                    1  2.321928   
                adipose tissue                               2  2.736966   
...                                                        ...       ...   
viral           viral infection                              4  2.000000   
                virus                                        5  1.514573   
viral entry     viral infection                              1  1.321928   
                virus                                        2  1.514573   
viral infection virus                                        4  1.514573   

                                                         npmi         @type  
@source_id      @target_id                                                   
ace inhibitor   acetaminophen                        0.537244  CoOccursWith  
                acute lung injury                    0.698970  CoOccursWith  
                acute respiratory distress syndrome  0.522879  CoOccursWith  
                adenosine                            0.537244  CoOccursWith  
                adipose tissue                       0.823909  CoOccursWith  
...                                                       ...           ...  
viral           viral infection                      0.861353  CoOccursWith  
                virus                                0.757287  CoOccursWith  
viral entry     viral infection                      0.305865  CoOccursWith  
                virus                                0.455932  CoOccursWith  
viral infection virus                                0.652291  CoOccursWith  

[9748 rows x 5 columns]

In [24]:
entity_nodes = graph.nodes_of_type("Entity").copy()

In [25]:
paper_frequency = mentions.groupby("entity").aggregate(set)["paper"].apply(len)
paper_frequency.name = "paper_frequency"

entity_nodes["paper_frequency"] = paper_frequency

In [26]:
paper_network = PandasPGFrame.from_frames(
    nodes=entity_nodes,
    edges=paper_cooccurrence_edges,
    node_prop_types={
        "paper_frequency": "numeric",
        "section_frequency": "numeric",
        "paragraph_frequency": "numeric"
    },
    edge_prop_types={
        "frequency": "numeric",
        "ppmi": "numeric",
        "npmi": "numeric"
    })

In [27]:
paper_network.edges(raw_frame=True)

common_factors  \
@source_id      @target_id                                                                      
ace inhibitor   acetaminophen                                                        {197804}   
                acute lung injury                                            {197804, 184360}   
                acute respiratory distress syndrome                          {197804, 184360}   
                adenosine                                                            {184360}   
                adipose tissue                                               {197804, 184360}   
...                                                                                       ...   
viral           viral infection                              {211125, 214924, 211373, 184360}   
                virus                                {179426, 184360, 211125, 214924, 211373}   
viral entry     viral infection                                                      {214924}   
                virus                                                        {214924, 179426}   
viral infection virus                                        {211125, 214924, 211373, 184360}   

                                                     frequency      ppmi  \
@source_id      @target_id                                                 
ace inhibitor   acetaminophen                                1  2.321928   
                acute lung injury                            2  2.321928   
                acute respiratory distress syndrome          2  1.736966   
                adenosine                                    1  2.321928   
                adipose tissue                               2  2.736966   
...                                                        ...       ...   
viral           viral infection                              4  2.000000   
                virus                                        5  1.514573   
viral entry     viral infection                              1  1.321928   
                virus                                        2  1.514573   
viral infection virus                                        4  1.514573   

                                                         npmi         @type  
@source_id      @target_id                                                   
ace inhibitor   acetaminophen                        0.537244  CoOccursWith  
                acute lung injury                    0.698970  CoOccursWith  
                acute respiratory distress syndrome  0.522879  CoOccursWith  
                adenosine                            0.537244  CoOccursWith  
                adipose tissue                       0.823909  CoOccursWith  
...                                                       ...           ...  
viral           viral infection                      0.861353  CoOccursWith  
                virus                                0.757287  CoOccursWith  
viral entry     viral infection                      0.305865  CoOccursWith  
                virus                                0.455932  CoOccursWith  
viral infection virus                                0.652291  CoOccursWith  

[9748 rows x 5 columns]

In [28]:
paper_network.nodes(raw_frame=True)

,@type,paper_frequency
@id,,
ace inhibitor,Entity,2
acetaminophen,Entity,2
acute lung injury,Entity,4
acute respiratory distress syndrome,Entity,6
adenosine,Entity,2
...,...,...
vildagliptin,Entity,2
viral,Entity,5
viral entry,Entity,2


### Section-based co-occurrence

In [29]:
def aggregate_sections(data):
    return set(sum(data["section"].apply(list), []))

In [31]:
section_cooccurrence_edges = gen.generate_from_edges(
     "OccursIn", 
    factor_aggregator=aggregate_sections,
    compute_statistics=["frequency", "ppmi", "npmi"])

Computing total factor instances...
Examining 12246 pairs of terms for co-occurrence...


In [32]:
section_cooccurrence_edges["@type"] = "CoOccursWith"
section_cooccurrence_edges

common_factors  \
@source_id      @target_id                                                                               
ace inhibitor   acetaminophen                        {197804:Management Of Children And Young Peopl...   
                acute lung injury                             {184360:Caption, 184360:Aceis And Arbs }   
                acute respiratory distress syndrome                                   {184360:Caption}   
                angioedema                           {184360:Combined Therapeutic Potential Targeti...   
                angiotensin ii receptor antagonist   {184360:Caption, 184360:Aceis And Arbs , 18436...   
...                                                                                                ...   
viral           viral entry                          {214924:Angiotensin-Converting Enzyme 2 Expres...   
                viral infection                      {214924:The Immune Response To Sars-Cov-2 , 21...   
                virus                                {214924:The Immune Response To Sars-Cov-2 , 18...   
viral entry     virus                                {179426:Role Of Dpp4 Enzyme And Dpp4 Inhibitor...   
viral infection virus                                {214924:The Immune Response To Sars-Cov-2 , 21...   

                                                     frequency      ppmi  \
@source_id      @target_id                                                 
ace inhibitor   acetaminophen                                1  2.557995   
                acute lung injury                            2  1.683526   
                acute respiratory distress syndrome          1  0.683526   
                angioedema                                   1  2.557995   
                angiotensin ii receptor antagonist           3  3.405992   
...                                                        ...       ...   
viral           viral entry                                  5  2.821030   
                viral infection                              6  2.821030   
                virus                                        9  1.821030   
viral entry     virus                                        2  1.236067   
viral infection virus                                        5  2.294961   

                                                         npmi         @type  
@source_id      @target_id                                                   
ace inhibitor   acetaminophen                        0.380206  CoOccursWith  
                acute lung injury                    0.293916  CoOccursWith  
                acute respiratory distress syndrome  0.101595  CoOccursWith  
                angioedema                           0.380206  CoOccursWith  
                angiotensin ii receptor antagonist   0.662263  CoOccursWith  
...                                                       ...           ...  
viral           viral entry                          0.640271  CoOccursWith  
                viral infection                      0.680922  CoOccursWith  
                virus                                0.511813  CoOccursWith  
viral entry     virus                                0.215797  CoOccursWith  
viral infection virus                                0.520873  CoOccursWith  

[5207 rows x 5 columns]

In [33]:
section_network = PandasPGFrame.from_frames(
    nodes=entity_nodes,
    edges=section_cooccurrence_edges,
    node_prop_types={
        "paper_frequency": "numeric",
        "section_frequency": "numeric",
        "paragraph_frequency": "numeric"
    },
    edge_prop_types={
        "frequency": "numeric",
        "ppmi": "numeric",
    })

### Paragraph-based co-occurrence

In [34]:
def aggregate_paragraphs(data):
    return set(sum(data["paragraph"].apply(list), []))

In [39]:
paragraph_cooccurrence_edges = gen.generate_from_edges(
     "OccursIn", 
    factor_aggregator=aggregate_paragraphs,
    compute_statistics=["frequency", "ppmi", "npmi"],
    parallelize=True, cores=8)

Computing total factor instances...
Examining 12246 pairs of terms for co-occurrence...


In [40]:
paragraph_cooccurrence_edges["@type"] = "CoOccursWith"
paragraph_cooccurrence_edges

common_factors  \
@source_id    @target_id                                                           
acetaminophen blood            {179426:Blood Glucose Monitoring ::: Special A...   
              covid-19         {197804:Discussion:51, 197804:Management Of Ch...   
              glucose          {179426:Blood Glucose Monitoring ::: Special A...   
              ibuprofen        {197804:Discussion:51, 197804:Management Of Ch...   
              insulin          {179426:Blood Glucose Monitoring ::: Special A...   
...                                                                          ...   
viral         viral infection  {211373:Introduction:5, 214924:The Interplay B...   
              virus            {214924:Abstract:1, 211373:Introduction:5, 184...   
vildagliptin  viral            {179426:Role Of Dpp4 Enzyme And Dpp4 Inhibitor...   
              viral entry      {179426:Role Of Dpp4 Enzyme And Dpp4 Inhibitor...   
              virus            {184360:Gliptins ::: Therapeutic Potential Of ...   

                               frequency      ppmi      npmi         @type  
@source_id    @target_id                                                    
acetaminophen blood                    1  1.321928  0.162613  CoOccursWith  
              covid-19                 2  0.000000  0.000000  CoOccursWith  
              glucose                  1  0.669851  0.082400  CoOccursWith  
              ibuprofen                2  5.129283  0.719467  CoOccursWith  
              insulin                  1  1.959358  0.241025  CoOccursWith  
...                                  ...       ...       ...           ...  
viral         viral infection          6  3.736966  0.674017  CoOccursWith  
              virus                    7  2.084889  0.391754  CoOccursWith  
vildagliptin  viral                    1  1.152003  0.141710  CoOccursWith  
              viral entry              1  3.222392  0.396393  CoOccursWith  
              virus                    3  2.669851  0.407965  CoOccursWith  

[2755 rows x 5 columns]

In [41]:
paragraph_network = PandasPGFrame.from_frames(
    nodes=entity_nodes,
    edges=paragraph_cooccurrence_edges,
    node_prop_types={
        "paper_frequency": "numeric",
    },
    edge_prop_types={
        "frequency": "numeric",
        "ppmi": "numeric",
        "npmi": "numeric"
    })

**NB**

_Positive pointwise mutual information (PPMI)_ is defined as $PPMI(x, y) = \log_2{\frac{p(x, y)}{p(x)p(y)}} $, if $p(x) \neq 0$ and $p(y) \neq 0$, and $PPMI(x, y) = 0$ otherwise.

_Normalized pointwise mutual information (NPMI)_ is defined as $NPMI(x, y) = \frac{PPMI(x, y)}{-\log_2{p(x, y)}} $.

The mutual-information-based distances are caluculated as $\frac{1}{(N/P)PMI}$.

### Create metric processor objects

In [42]:
import math

def compute_distance(x):
    return 1 / x if x > 0 else math.inf

In [43]:
npmi_distance = paper_network.edges(raw_frame=True)["npmi"].apply(compute_distance)
npmi_distance.name = "distance_npmi"
paper_network.add_edge_properties(npmi_distance, "numeric")

In [44]:
paper_network.edges(raw_frame=True)

common_factors  \
@source_id      @target_id                                                                      
ace inhibitor   acetaminophen                                                        {197804}   
                acute lung injury                                            {197804, 184360}   
                acute respiratory distress syndrome                          {197804, 184360}   
                adenosine                                                            {184360}   
                adipose tissue                                               {197804, 184360}   
...                                                                                       ...   
viral           viral infection                              {211125, 214924, 211373, 184360}   
                virus                                {179426, 184360, 211125, 214924, 211373}   
viral entry     viral infection                                                      {214924}   
                virus                                                        {214924, 179426}   
viral infection virus                                        {211125, 214924, 211373, 184360}   

                                                     frequency      ppmi  \
@source_id      @target_id                                                 
ace inhibitor   acetaminophen                                1  2.321928   
                acute lung injury                            2  2.321928   
                acute respiratory distress syndrome          2  1.736966   
                adenosine                                    1  2.321928   
                adipose tissue                               2  2.736966   
...                                                        ...       ...   
viral           viral infection                              4  2.000000   
                virus                                        5  1.514573   
viral entry     viral infection                              1  1.321928   
                virus                                        2  1.514573   
viral infection virus                                        4  1.514573   

                                                         npmi         @type  \
@source_id      @target_id                                                    
ace inhibitor   acetaminophen                        0.537244  CoOccursWith   
                acute lung injury                    0.698970  CoOccursWith   
                acute respiratory distress syndrome  0.522879  CoOccursWith   
                adenosine                            0.537244  CoOccursWith   
                adipose tissue                       0.823909  CoOccursWith   
...                                                       ...           ...   
viral           viral infection                      0.861353  CoOccursWith   
                virus                                0.757287  CoOccursWith   
viral entry     viral infection                      0.305865  CoOccursWith   
                virus                                0.455932  CoOccursWith   
viral infection virus                                0.652291  CoOccursWith   

                                                     distance_npmi  
@source_id      @target_id                                          
ace inhibitor   acetaminophen                             1.861353  
                acute lung injury                         1.430677  
                acute respiratory distress syndrome       1.912489  
                adenosine                                 1.861353  
                adipose tissue                            1.213727  
...                                                            ...  
viral           viral infection                           1.160964  
                virus                                     1.320504  
viral entry     viral infection                           3.269412  
                virus                    

In [45]:
npmi_distance = section_network.edges(raw_frame=True)["npmi"].apply(compute_distance)
npmi_distance.name = "distance_npmi"
section_network.add_edge_properties(npmi_distance, "numeric")

In [46]:
npmi_distance = paragraph_network.edges(raw_frame=True)["npmi"].apply(compute_distance)
npmi_distance.name = "distance_npmi"
paragraph_network.add_edge_properties(npmi_distance, "numeric")

Density of a network is quantified by the proportion of all possible edges ($n(n-1) / 2$ for the undirected network with $n$ nodes) that are realized.

In [47]:
paper_metrics = GTMetricProcessor(paper_network)
section_metrics = GTMetricProcessor(section_network)
paragraph_metrics = GTMetricProcessor(paragraph_network)

In [48]:
print("Density of the paper-based network: ", paper_metrics.density())
print("Density of the section-based network: ", section_metrics.density())
print("Density of the paragraph-based network: ", paragraph_metrics.density())

Density of the paper-based network:  0.7960150253143884
Density of the section-based network:  0.42520006532745386
Density of the paragraph-based network:  0.22497141923893516


The results above show that in the paper, section and paragraph network repsectively 80%, 42% and 22% of all possible term pairs co-occur at least once.

## Nearest neighours by (N/P)MI

To illustrate the importance of computing mutual-information-based scores over raw frequencies consider the following example, where we would like to estimate top closest (most related) neighbors to a specific term.

We will use the paragraph-based network and the raw co-occurrence frequency as the weight of our co-occurrence relation.

In [49]:
paragraph_path_finder = GTPathFinder(paragraph_network)

In [50]:
paragraph_path_finder.top_neighbors("glucose", 10, weight="frequency")

{'insulin': 11.0,
 'hyperglycemia': 8.0,
 'infectious disorder': 6.0,
 'inflammation': 6.0,
 'sars coronavirus': 6.0,
 'interleukin-6': 5.0,
 'sars-cov-2': 5.0,
 'glucose metabolism disorder': 4.0,
 'glyburide': 4.0,
 'oral cavity': 4.0}

In [51]:
paragraph_path_finder.top_neighbors("lung", 10, weight="frequency")

{'sars-cov-2': 13.0,
 'pulmonary': 10.0,
 'sars coronavirus': 10.0,
 'viral': 9.0,
 'mouse': 7.0,
 'virus': 5.0,
 'vascular': 4.0,
 'viral infection': 4.0,
 'middle east respiratory syndrome coronavirus': 3.0,
 'survival': 3.0}

We observe that 'glucose' and 'lung' share a lot of the closest neighbors by raw frequency. If we look into the list of top 10 entities by paragraph frequency in the entire corpus and we notice that 'glucose' and 'blood' co-occur the most with the terms that are simply the most frequent in our corpus, such as 'covid-19' and 'diabetes mellitus'.

(Closest inspection of the distribution of weighted node degrees suggests that the network contains _hubs_, nodes with significantly high-degree connectivity to other nodes.)

In [53]:
paragraph_network._nodes

,@type,paper_frequency
@id,,
ace inhibitor,Entity,2
acetaminophen,Entity,2
acute lung injury,Entity,4
acute respiratory distress syndrome,Entity,6
adenosine,Entity,2
...,...,...
vildagliptin,Entity,2
viral,Entity,5
viral entry,Entity,2


In [59]:
paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["paper_frequency"])

,@type,paper_frequency
@id,,
covid-19,Entity,20
diabetes mellitus,Entity,19
coronavirus,Entity,16
glucose,Entity,13
death,Entity,9
glyburide,Entity,9
infectious disorder,Entity,9
blood,Entity,8
hyperglycemia,Entity,8


To account for the presence of such hubs, we use the mutual-information-based scores presented above. They 'balance' the influence of the highly connected hub nodes such as 'covid-19' and 'diabetes mellitus' in our example.

In [60]:
paragraph_path_finder.top_neighbors("glucose", 10, weight="npmi")

{'glucose metabolism disorder': 0.43558951200762297,
 'insulin': 0.41957609533629175,
 'thrombophilia': 0.4079646453270325,
 'insulin infusion': 0.3744908698338857,
 'leukopenia': 0.3744908698338857,
 'millimole per liter': 0.3744908698338857,
 'troponin t, cardiac muscle': 0.3744908698338857,
 'hyperglycemia': 0.3255525953220345,
 'serum ferritin': 0.2953531691104843,
 'tissue': 0.2924401924613069}

In [61]:
paragraph_path_finder.top_neighbors("lung", 10, weight="npmi")

{'pulmonary': 0.6362945400636919,
 'receptor binding': 0.46595542454855043,
 'viral infection': 0.4465392749200213,
 'mouse': 0.4362945258726772,
 'viral': 0.4233482775367079,
 'sars coronavirus': 0.4042589954647716,
 'survival': 0.39499326084547054,
 'm protein': 0.2856252008999382,
 'myocardium': 0.2856252008999382,
 'neoplasm': 0.2856252008999382}

## Node centralities

In this example we will compute the Degree and PageRank centralities only for the raw frequency, and the Betweenness centrality for the mutual-information-based scores.

_Degree centrality_ is given by the sum of weights of all incident edges of the given node and characterizes the importance of the node in the network in terms of its connectivity to other nodes (high degree = high connectivity).

In [62]:
paragraph_metrics.degree_centrality("frequency", write=True, write_property="degree")

_PageRank centrality_ is another measure that estimated the importance of the given node in the network. Roughly speaking it can be interpreted as the probablity that having landed on a random node in the network we will jump to the given node (here the edge weights are taken into account").

https://en.wikipedia.org/wiki/PageRank

In [63]:
paragraph_metrics.pagerank_centrality("frequency", write=True, write_property="pagerank")

We then compute the betweenness centrality based on the NPMI distances.

_Betweenness centrality_ is a node importance measure that estimates how often a shortest path between a pair of nodes will pass through the given node.

In [64]:
paragraph_metrics.betweenness_centrality("distance_npmi", write=True, write_property="betweenness")

In [65]:
new_paragraph_network = paragraph_metrics.get_pgframe()

In [66]:
new_paragraph_network.nodes(raw_frame=True)

,@type,paper_frequency,degree,pagerank,betweenness
@id,,,,,
ace inhibitor,Entity,2.0,47.0,0.001907,0.000000
acetaminophen,Entity,2.0,7.0,0.001907,0.000000
acute lung injury,Entity,4.0,132.0,0.001942,0.000289
acute respiratory distress syndrome,Entity,6.0,149.0,0.001957,0.000951
adenosine,Entity,2.0,23.0,0.001918,0.001323
...,...,...,...,...,...
vildagliptin,Entity,2.0,5.0,0.017599,0.000331
viral,Entity,5.0,18.0,0.069727,0.004880
viral entry,Entity,2.0,1.0,0.031216,0.000000


In [67]:
print("Top 10 nodes by degree")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["degree"]).index:
    print("\t", n)

Top 10 nodes by degree
	 covid-19
	 diabetes mellitus
	 angiotensin-converting enzyme 2
	 coronavirus
	 dipeptidyl peptidase 4
	 blood
	 acute respiratory distress syndrome
	 glucose
	 death
	 acute lung injury


In [68]:
print("Top 10 nodes by PageRank")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["pagerank"]).index:
    print("\t", n)

Top 10 nodes by PageRank
	 virus
	 viral
	 sars-cov-2
	 viral entry
	 viral infection
	 vascular
	 sars coronavirus
	 tissue
	 vildagliptin
	 sitagliptin


In [69]:
print("Top 10 nodes by betweenness")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["betweenness"]).index:
    print("\t", n)

Top 10 nodes by betweenness
	 glucose metabolism disorder
	 lung
	 interleukin-6
	 kidney
	 glyburide
	 inflammation
	 interleukin-19
	 sars-cov-2
	 blood
	 infectious disorder


## Compute multiple metrics in one go

Alternatively, we can compute all the metrics in one go. To do so, we need to specify edge attributes used for computing different metrics (if an empty list is specified as a weight list for a metric, computation of this metric is not performed). 

We select the paragraph-based network and re-compute all some of the previously illustrated metrics as follows:

In [70]:
result_metrics = paragraph_metrics.compute_all_node_metrics(
    degree_weights=["frequency"],
    pagerank_weights=["frequency"],
    betweenness_weights=["distance_npmi"])

In [71]:
result_metrics

{'degree': {'frequency': {'ace inhibitor': 47.0,
   'acetaminophen': 7.0,
   'acute lung injury': 132.0,
   'acute respiratory distress syndrome': 149.0,
   'adenosine': 23.0,
   'adipose tissue': 28.0,
   'angioedema': 22.0,
   'angiotensin ii receptor antagonist': 82.0,
   'angiotensin-1': 71.0,
   'angiotensin-2': 17.0,
   'angiotensin-converting enzyme': 10.0,
   'angiotensin-converting enzyme 2': 329.0,
   'animal': 41.0,
   'apoptosis': 19.0,
   'bals r': 6.0,
   'basal': 34.0,
   'blood': 156.0,
   'blood vessel': 28.0,
   'bradykinin': 15.0,
   'c-c motif chemokine 1': 41.0,
   'c-reactive protein': 32.0,
   'cardiac failure': 14.0,
   'cardiovascular disorder': 98.0,
   'cardiovascular system': 95.0,
   'cd44 antigen': 40.0,
   'cellular secretion': 45.0,
   'cerebrovascular': 17.0,
   'chemokine': 40.0,
   'chest pain': 20.0,
   'chloroquine': 8.0,
   'chronic disease': 8.0,
   'chronic kidney disease': 18.0,
   'comorbidity': 13.0,
   'confounding factors': 22.0,
   'coronav

## Community detection

_Community detection_ methods partition the network into clusters of densely connected nodes in a way that nodes in the same community are more connected between themselves relatively to the nodes in different communities.

The _modularity_ measure quantifies how well the communities are 'separated' (https://en.wikipedia.org/wiki/Modularity_(networks)).

In [72]:
# print("Paper-based graph")
# print("-----------------")
# _ = detect_communities(paper_comention_network, weight="frequency", set_attr="community")
# _ = detect_communities(paper_comention_network, weight="npmi", set_attr="community_npmi")
# print()

# print("Section-based graph")
# print("-------------------")
# _ = detect_communities(section_comention_network, weight="frequency", set_attr="community")
# _ = detect_communities(section_comention_network, weight="npmi", set_attr="community_npmi")
# print()

# print("Paragraph-based graph")
# print("---------------------")
# _ = detect_communities(paragraph_comention_network, weight="frequency", set_attr="community")
# _ = detect_communities(paragraph_comention_network, weight="npmi", set_attr="community_npmi")
# print()

In the example above we can see that the best partition is performed on the paragraph-based network using the NPMI edge weight (resulting in the modularity 0.34).

## Export network and the computed metrics

In [73]:
# # Save graph nodes as a pickled pandas.DataFrame
# save_network(paragraph_comention_network, "data/literature_comention")

In [74]:
# Save the graph for Gephi import.
new_paragraph_network.export_to_gephi(
    "data/gephi_literature_comention", 
    node_attr_mapping = {
        "degree": "Degree",
        "pagerank": "PageRank",
        "betweenness": "Betweenness",
#         "community_npmi": "Community"
    },
    edge_attr_mapping={
        "npmi": "Weight"
    })

The representation of the network saved above can be imported into Gephi for producing graph visualizations, as in the following example:

In the figures below colors represent communities detected using the NPMI weight, node sizes are proportional to the PageRank of nodes and edge thickness to the NPMI values.

**Full network**
<img src="./figures/literature/full_network.png" alt="Literature co-occurrence network" style="width: 400px;"/>

**Community "Symptoms and comorbidities"**
<img src="./figures/literature/covid_19_comorbidities.png" alt="Drawing" style="width: 400px;"/>

**Community "Viral biology"**
<img src="./figures/literature/virus.png" alt="Drawing" style="width: 600px;"/>

**Community "Immunity"**
<img src="./figures/literature/immunity.png" alt="Drawing" style="width: 600px;"/>

## Minimum spanning trees

A _minimum spanning tree_ of a network is given by a subset of edges that make the network connected ($n - 1$ edges connecting $n$ nodes). Its weighted version minimizes not only the number of edges included in the tree, but the total edge weight.

In the following example we compute a minimum spanning tree minimizing the NPMI-based distance weight of the network edges.

In [112]:
gt_paragraph_path_finder = GTPathFinder(new_paragraph_network)

In [113]:
tree = graph_tool_to_pgframe(gt_paragraph_path_finder.minimum_spanning_tree(distance="distance_npmi"))

In [114]:
tree.export_to_gephi(
    "data/gephi_literature_spanning_tree", 
    node_attr_mapping = {
        "degree": "Degree",
        "pagerank": "PageRank",
        "betweenness": "Betweenness",
#         "community_npmi": "Community"
    },
    edge_attr_mapping={
        "npmi": "Weight"
    })

The representation of the network saved above can be imported into Gephi for producing graph visualizations, as in the following example:

In the figures below colors represent communities detected using the NPMI weight, node sizes are proportional to the PageRank of nodes and edge thickness to the NPMI values.

**Full network**
<img src="./figures/literature/tree.png" alt="Literature co-occurrence MST" style="width: 400px;"/>

**Zoom into "covid-19"**
<img src="./figures/literature/tree_covid-19.png" alt="Drawing" style="width: 700px;"/>

## Shortest path search

The _shortest path search problem_ consisits in finding a sequence of edges from the source node to the target node that minimizes the cumulative weight (or distance) associated to the edges. 

In [115]:
path = gt_paragraph_path_finder.shortest_path("lung", "sars-cov-2")
pretty_print_paths([path])

lung <->  <-> sars-cov-2
         


The cell above illustrates that the single shortest path form 'lung' and 'sars-cov-2' consists of the direct edge between them.

We adapt this problem to the literature exploration task, i.e. having fixed the source and the target concepts (the relation here is actually symmetric as the edges of our network are undirected), we would like to find a _set_ of $n$ shortest paths between them. Moreover, we would like these paths to be _indirect_ (not to include the direct edge from the source to the target). In the following examples we use mutual-information-based edge weights to perform our literature exploration. 

The library includes two strategies for finding such $n$ shortest paths. The first strategy uses Yen's algorithm for finding $n$ loopless shortest paths from the source to the target (https://en.wikipedia.org/wiki/Yen%27s_algorithm).


In [116]:
nx_paragraph_path_finder = NXPathFinder(new_paragraph_network)
paths = nx_paragraph_path_finder.n_shortest_paths(
    "lung", "sars-cov-2", n=10,
    distance="distance_npmi", strategy="yen")

In [117]:
pretty_print_paths(paths)

lung <->                                 <-> sars-cov-2
         
         acute lung injury
         pulmonary
         receptor binding
         human
         viral
         angiotensin-converting enzyme 2
         host cell
         dna replication
         acute lung injury <-> pulmonary


The second, _naive_, strategy is suitable in the scenarios when our networks are large and highly dense (then the performance of Yen's algorithm degragates as the number of edges is approaching $O(N^2)$ with $N$ being the number of nodes). 

This strategy simply finds _all_ the indirect shortest paths from the source to the target (in dense graphs the most common such paths are of length 2, i.e. `source <-> intermediary <-> target`, and therefore, the number of such path is roughly proportional to the number of nodes in the network). Then, the cumulative distance score is computed for every path and the top $n$ paths with the best score are selected.

In [118]:
paths = gt_paragraph_path_finder.n_shortest_paths(
    "lung", "sars-cov-2", n=10,
    distance="distance_npmi", strategy="naive")

In [119]:
pretty_print_paths(paths)

lung <->                                 <-> sars-cov-2
         
         acute lung injury
         pulmonary
         receptor binding
         human
         viral
         angiotensin-converting enzyme 2
         host cell
         dna replication
         human kidney organoids


We can also search for a path in the previously computed spanning tree

In [120]:
tree_path_finder = GTPathFinder(tree)
path = tree_path_finder.shortest_path("lung", "sars-cov-2")
print(path)

('lung', 'acute lung injury', 'animal', 'mouse', 'human dpp4', 'human immunodeficiency virus', 'coronaviridae', 'sars-cov-2')


The library provides an additional utility for finding _tripaths_, paths of the shape `source <-> intermediary <-> target`. Setting the parameter `intersecting` to `False` we can ensure that the entities the sets of entities discovered on the paths `source <-> intermediary` and `intermediary <-> target` do not overlap.

In [121]:
path_a_b, path_b_c = gt_paragraph_path_finder.n_shortest_tripaths(
    "lung", "glucose", "sars-cov-2", 10,
    strategy="naive", distance="distance_npmi", overlap=False)

In [122]:
pretty_print_tripaths("lung", "glucose", "sars-cov-2", 10, path_a_b, path_b_c)

lung ->                                -> glucose ->                   -> sars-cov-2
        inflammation                                   cerebrovascular
        islet of langerhans                            plasmid
        serum                                          basal
        oral cavity                                    coronavirus
        death                                          headache
        middle east respiratory syndrome               comorbidity
        neutrophil                                     infectious disorder
        sulfonylurea antidiabetic agent                viral entry
        therapeutic corticosteroid                     sars coronavirus
        chemokine                                      person


## Nested path search

To explore the space of co-occurring terms in depth, we can run the path search procedure presented above in a _nested fashion_. For each edge $e_1, e_2, ..., e_n$ encountered on a path from the source to the target  from, we can
further expand it into $n$ shortest paths between each pair of successive entities (i.e. paths between $e_1$ and $e_2$, $e_2$ and $e_3$, etc.).

In [123]:
paths = gt_paragraph_path_finder.n_nested_shortest_paths(
    "lung", "glucose", top_level_n=10, nested_n=2, depth=2, distance="distance_npmi",
    strategy="naive")

In [124]:
paths

[('inflammation', 'glucose metabolism disorder', 'glucose'),
 ('neutrophil', 'lymphocyte', 'glucose'),
 ('inflammation', 'thrombophilia', 'glucose'),
 ('lung', 'serum', 'inflammation'),
 ('lung', 'oral cavity', 'glucose'),
 ('chemokine', 'growth factor', 'glucose'),
 ('neutrophil', 'millimole per liter', 'glucose'),
 ('therapeutic corticosteroid', 'oral cavity', 'glucose'),
 ('islet of langerhans', 'hyperglycemia', 'glucose'),
 ('lung', 'chemokine', 'glucose'),
 ('sulfonylurea antidiabetic agent', 'blood', 'glucose'),
 ('lung', 'sulfonylurea antidiabetic agent', 'glucose'),
 ('lung', 'serum', 'glucose'),
 ('middle east respiratory syndrome', 'lymphocyte', 'glucose'),
 ('chemokine', 'glucose metabolism disorder', 'glucose'),
 ('therapeutic corticosteroid', 'insulin', 'glucose'),
 ('lung', 'inflammation'),
 ('serum', 'thrombophilia', 'glucose'),
 ('death', 'leukopenia', 'glucose'),
 ('lung', 'oral cavity'),
 ('middle east respiratory syndrome', 'glucose'),
 ('oral cavity', 'glucose'),
 (

We can now visualize the subnetwork constructed using the nodes and the edges discovered during our nested path search.

In [125]:
nodes, edges = graph_elements_from_paths(paths)

In [126]:
summary_graph = graph_tool_to_pgframe(
    gt_paragraph_path_finder.get_subgraph(
        nodes_to_exclude=[
            n for n in gt_paragraph_path_finder.get_nodes()
            if n not in nodes
        ],
        edges_to_exclude=[
            (s, t) for s, t in gt_paragraph_path_finder.get_edges()
            if (s, t) not in edges
        ]))

In [127]:
print("Number of nodes: ", summary_graph.number_of_nodes())
print("Number of edges: ", summary_graph.number_of_edges())

Number of nodes:  27
Number of edges:  23


In [128]:
# Save the graph for Gephi import.
summary_graph.export_to_gephi(
    "data/gephi_literature_path_graph", 
    node_attr_mapping = {
        "degree": "Degree",
        "pagerank": "PageRank",
        "betweenness": "Betweenness",
#         "community_npmi": "Community"
    },
    edge_attr_mapping={
        "npmi": "Weight"
    })

The resulting graph visualized with Gephi
<img src="./figures/literature/path_graph.png" alt="Literature co-occurrence network" style="width: 800px;"/>